In [5]:
import scapy.all as sp
import pandas as pd
import os
import sys
import random
import math
import numpy as np
import random

In [6]:
def determining_normal_rate_packets(pcap_file):
    time_start = math.inf
    time_end = 0
    total_packets = 0
    total_rate = 0
    with sp.PcapReader(pcap_file) as trace:
        for packet in trace:
            total_packets += 1
            time = packet.time
            time_start = min(time_start, time)
            time_end = max(time, time_end)
            total_rate += len(packet)
        total_time = time_end - time_start

    return float(total_packets/total_time)

    


In [7]:
#### simulating a real life scenario
def arrange_by_arrival_time(pcap_file):
    packets = []
    times = []
    with sp.PcapReader(pcap_file) as trace:
        for packet in trace:
            packets.append(packet)
            times.append(packet.time)

    sorted_idx = np.argsort(times)
    return_packets = []
    for i in sorted_idx:
        return_packets.append(packets[i])
    
    
    return return_packets
    
   
    

In [8]:
def simulate_by_packets(ave_packets_rate, packets_sorted, fluctuation_rate, queue_size, delay_time):
    return_packets = []
    ave_packets_rate = ave_packets_rate
    injected_packets_num = []
    #### fluctuation rate determines the upper and lower bound the number of packets per second should be
    ##### if the number of packets per second goes greater than the upper bound, then some packets will be queued
    ##### if the number of packets per second is smaller than the lower bound, then packets will be taken from  dynamic queue
    #### first, then the static queue(fake packets)
    dbs = []
    cbs = []
    cbs.append(packets_sorted[0].copy())

    
    start_time = math.floor(packets_sorted[0].time)
    end_time = math.ceil(packets_sorted[-1].time)
    index = 0
    num_packets = len(packets_sorted)

    cbs.append(packets_sorted[0])
    for time in range(start_time, end_time):
        print("time")
        print(time-start_time)
        num_injected = 0 ## number of injected packets from time to time +1
        udp = []
        non_udp = []
        while index<num_packets:
            packet = packets_sorted[index]
            ##### only considering the packets that whose time lie between time and time +1. 
            ##### This resembles real life situation that the packets are incoming
            if packet.time < time+1:
                if packet.haslayer(sp.UDP):
                    udp.append(packet)
                else:
                    non_udp.append(packet)
                index +=1
            else:
                break
        print("udp")
        print(len(udp))
        
        
        #### We make the assumption here that udp packets are urgent and need to be sent immediately so we do not introduce delays for udp
        return_packets = return_packets + udp 
        num_injected += len(udp)
        
        ####### queuing the non udp packets
        dbs = dbs+non_udp      
        #### we are assuming that the non_udp packets cannot wait for more than delay_time seconds, 
        ####after delay_time second, any outstanding packets have to be sent  
        pop_packets = 0
        for packet in dbs:
            if time - packet.time > delay_time:
                random_time = time + random.uniform(0,1)
                packet.time = random_time
                return_packets.append(packet)
                pop_packets+=1
                num_injected +=1
            else:
                break
        for i in range(pop_packets):
            dbs.pop(0)
            
        print("udp and urgent")
        print(num_injected)
                
#         ###### we decide if we should do some more injection, 
        ####### if the num_injected has not exceeded the upper threshold, we may inject more traffic
        if num_injected < ave_packets_rate*(1+fluctuation_rate):
        ############ it is a big decision how much to inject so that it both decrease the buffer as well as generate randomness#########

#             ##### we decide randomly how much to inject, 
            #######in doing this, we also take into account the length of the queue size
            injected_dbs = random.randint(math.floor(ave_packets_rate*(1-fluctuation_rate)),math.floor(ave_packets_rate))  - num_injected  
            
            if injected_dbs>0:
                print("random injection")
                print(injected_dbs)
                for i in range(injected_dbs):
                    if len(dbs)>0:
                        packet = dbs[0]
                        random_time = time + random.uniform(0,1)
                        packet.time = random_time
                        return_packets.append(packet)
                        dbs.pop(0)
                        num_injected +=1
                    else:
                        break

                
#             #### if the buffer is too full, we will inject more
        if len(dbs)>0:
            if len(dbs)> queue_size * ave_packets_rate:
                print("need more injection")
                injected_dbs = min(len(dbs)-int(queue_size * ave_packets_rate),math.floor(ave_packets_rate*(1+fluctuation_rate))-num_injected) 

            else:
                injected_dbs = int(queue_size * len(dbs))
            if injected_dbs>0:
                for i in range(injected_dbs):
                    if len(dbs)>0:
                        packet = dbs[0]
                        random_time = time + random.uniform(0,1)
                        packet.time = random_time
                        return_packets.append(packet)
                        dbs.pop(0)
                        num_injected +=1
                    else:
                        break
                print(injected_dbs)
        if num_injected <ave_packets_rate*(1-fluctuation_rate):
            injected_cbs = random.randint(int(ave_packets_rate*(1-fluctuation_rate)), int (ave_packets_rate)) - num_injected
            for i in range(injected_cbs):
                    packet = cbs[0].copy()
                    random_time = time + random.uniform(0,1)
                    packet.time = random_time
                    return_packets.append(packet)
                    num_injected +=1




                    
        injected_packets_num.append(num_injected)
#         print(len(total))
#         print(num_injected)
                
       
    return injected_packets_num, return_packets

In [9]:
def evaluate_buffer(ave_packets, injected_packets, buffer_level, fluctuation_rate):
    overhead_level = (sum(injected_packets)/len(injected_packets) - ave_packets)/ave_packets
    overshoot = 0
    for i in injected_packets:
        if i>buffer_level*ave_packets*(1+fluctuation_rate):
            overshoot+=1
    masking_score = (len(injected_packets) - overshoot)/len(injected_packets)
    return overhead_level, masking_score
    

In [22]:
def main():
    
    ## getting the average number of packets and average rate from previous time period or day, we will try to model the current period to be the same as the previous period
    ave_packets = determining_normal_rate_packets("/Users/vantran/Desktop/GRAD_RESEARCH/Shinan/IOT-data/data-clean/alexa/play_music/2/play_music_2_1614040090.pcap")
    print(ave_packets)
    ###### arranging the incoming packets by arrival time to simulate a real life scenario
    packets_sorted = arrange_by_arrival_time("/Users/vantran/Desktop/GRAD_RESEARCH/Shinan/IOT-data/data-clean/alexa/play_music/2/play_music_1_1614040019.pcap")
        
    ##### if we want to increase the level of padding to ensure more protection, we can increase the ave_packets as well as decrease the fluctuation rate
    buffer_level = 1.9
    fluctuation_rate = 0.3
    queue_size = 0.4
    delay_time = 6
#      ####### Van will vary the constants to see what will be the effect of varying these variables
    
    
    injected_packets, return_packets = simulate_by_packets(buffer_level*ave_packets, packets_sorted, fluctuation_rate,queue_size,delay_time)
    overhead_level, masking_score = evaluate_buffer(ave_packets, injected_packets, buffer_level, fluctuation_rate)
    print(overhead_level, masking_score)
    
    os.system("echo Writing out buffered pcaps...")
    sp.wrpcap('result.pcap', return_packets)
    os.system("echo Reordering Pcap...")
    os.system("reordercap -n {} {}".format("result.pcap","result_sorted_V.pcap"))
   

    
    
   
    

    

    
    


In [23]:
main()

152.69126246967048
time
0
udp
0
udp and urgent
0
random injection
274
time
1
udp
108
udp and urgent
108
random injection
103
time
2
udp
114
udp and urgent
114
random injection
91
time
3
udp
189
udp and urgent
189
random injection
73
9
time
4
udp
172
udp and urgent
172
random injection
113
36
time
5
udp
145
udp and urgent
145
random injection
116
24
time
6
udp
107
udp and urgent
107
random injection
172
10
time
7
udp
102
udp and urgent
102
random injection
104
need more injection
171
time
8
udp
113
udp and urgent
113
random injection
99
need more injection
165
time
9
udp
90
udp and urgent
90
random injection
144
need more injection
143
time
10
udp
58
udp and urgent
58
random injection
147
need more injection
162
time
11
udp
61
udp and urgent
61
random injection
211
time
12
udp
173
udp and urgent
173
random injection
33
13
time
13
udp
76
udp and urgent
76
random injection
161
time
14
udp
72
udp and urgent
72
random injection
188
time
15
udp
92
udp and urgent
92
random injection
163
time
